In [ ]:
import os
import pandas as pd

In [ ]:
data_dir = 'uspto_sent_compression'
file_names = os.listdir(data_dir)

In [ ]:
dfs = []
column_names = ['id','title','abstract','inv_org','inv_names','cpc']
for name in file_names:
  file_path = os.path.join(data_dir,name)
  df = pd.read_csv(file_path, sep='\t', lineterminator='\n',header=None,names=column_names)
  dfs.append(df)
df_uspto = pd.concat(dfs,ignore_index=True, sort=False)

In [ ]:
df_uspto.shape

(12481, 6)

In [ ]:
import pickle

In [ ]:
!pip install stanza

In [ ]:
os.environ["CORENLP_HOME"] = 'PATH-TO-CORENLP'

In [ ]:
import stanza
from stanza.server import CoreNLPClient
import time

client = CoreNLPClient(timeout=150000000, be_quiet=True, annotators=['parse'], 
endpoint='http://localhost:9001')
client.start()

time.sleep(10)

2022-05-31 10:50:15 INFO: Writing properties to tmp file: corenlp_server-99e35faecef4445f.props
2022-05-31 10:50:15 INFO: Starting server with command: java -Xmx5G -cp /content/drive/MyDrive/Colab Notebooks/thesis_PQAI/verbosity_model/stanford-corenlp-4.4.0/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 150000000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-99e35faecef4445f.props -annotators parse -preload -outputFormat serialized


In [ ]:
!pip install pycorenlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pycorenlp: filename=pycorenlp-0.3.0-py3-none-any.whl size=2145 sha256=b14c5394ef91ebc9a23635b6f0772d9a3d6546adcdcc2a5f2686696ae2d996dd
  Stored in directory: /root/.cache/pip/wheels/83/d8/ad/6b2276343ac605ee47e6beddb28331e96377909e5c816539c3
Successfully built pycorenlp


In [ ]:
from pycorenlp import StanfordCoreNLP
import json
from nltk import Tree

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
df_uspto.values.shape

(12481, 6)

In [ ]:
nlp = StanfordCoreNLP('http://localhost:9001')
start_index = 10367
tree_dir_path = 'trees/'
for index,val in enumerate(df_uspto.values[start_index:]):
  dict_trees = {}
  print('\n')
  print(start_index+index,'.')
  print("Title: ",val[1])
  

  abstract = val[2].replace(';','.')
  abstract = abstract.replace(',','.')
  abs_sents = sent_tokenize(abstract)
  
  #title parse tree
  title_out = nlp.annotate(val[1], properties={'annotators': 'parse','outputFormat': 'json','timeout': 400000})
  #print(type(title_out))
  #obj_title = json.loads(title_out)
  title_parse_tree =  title_out['sentences'][0]['parse']
  dict_trees['title_tree'] = title_parse_tree
  
  #abstract parse tree
  print('Abstract Sentences: ')
  dict_trees['abstract_tree'] = []
  for sent in abs_sents:
    print(sent)
    abs_out = nlp.annotate(sent, properties={'annotators': 'parse','outputFormat': 'json','timeout': 400000})
    
    
    abs_parse_tree =  abs_out['sentences'][0]['parse']
    dict_trees['abstract_tree'].append(abs_parse_tree)
  
  #write trees to pickle file
  with open(tree_dir_path+str(start_index+index)+'.pickle', 'wb') as handle:
                pickle.dump(dict_trees, handle, protocol=pickle.HIGHEST_PROTOCOL)

Streaming output truncated to the last 5000 lines.

12091 .
Title:  Fitment with valve and ethylene/a-olefin multi-block copolymer
Abstract Sentences: 
The present disclosure provides a fitment.
In an embodiment.
a fitment is provided and includes a top portion.
a base.
and a channel extending through the top portion and the base for passage of a flowable material.
The fitment includes a flexible valve extending across the channel.
The flexible valve includes a slit which opens to permit flow therethrough.
The flexible valve includes an ethylene/.alpha.-olefin multi-block copolymer.


12092 .
Title:  Method for obtaining self-adhesive booklet-label tape, without backing and protection tape
Abstract Sentences: 
The present abstract relates to a patent of invention for a method for obtaining a tape (1) for self-adhesive.
linerless leaflet labels.
pertaining to the field of self-adhesive articles.
comprising: 1)--Providing and unwinding a self-adhesive-material tape (20).
2)--Printing lab

In [ ]:
tree_f = os.listdir('trees')

In [ ]:
tree_f[-1]

'12480.pickle'